In [1]:
import pandas as pd

In [4]:
expenseFilePath = 'productionExpensesRaw.xlsx'
expenseDF = pd.read_excel(expenseFilePath, sheet_name='United States', header=3)

In [7]:
# Clean up column headers
expenseDF.columns = expenseDF.columns.map(str)
expenseDF.rename(columns={expenseDF.columns[0]: 'cat'}, inplace=True)
expenseDF.drop([0, 1, 2], axis=0, inplace=True)

In [10]:
# Reset index to treat 'cat' as a regular column
expenseDF.reset_index(drop=True, inplace=True)
expenseDF.columns = expenseDF.columns.map(lambda x: x[:4]) # only take the first 4 chars
# Remove footnotes (e.g., 1/) from category names
expenseDF['cat'] = expenseDF['cat'].astype(str).str.replace(r'\d+/', '', regex=True).str.strip().str.lower()

In [17]:
# Track parent category
parent_cat = None
cleaned_rows = []

for _, row in expenseDF.iterrows():
    cat = row['cat']
    data = row.drop('cat')

    # If entire data row is NaN, it's a parent category header
    if data.isna().all():
        parent_cat = cat
    else:
        full_cat = f"{parent_cat} > {cat}" if parent_cat else cat
        cleaned_rows.append({'cat': full_cat, **data})
        
# Recreate cleaned DataFrame
cleaned_expenseDF = pd.DataFrame(cleaned_rows)

In [20]:
# Convert all data columns to numeric and multiply by 1000
for col in cleaned_expenseDF.columns[1:]:
    cleaned_expenseDF[col] = pd.to_numeric(cleaned_expenseDF[col], errors='coerce') * 1000

print(cleaned_expenseDF.head(5))

                                                cat          2016  \
0                      including operator dwellings  349938198000   
1                      excluding operator dwellings  345623632000   
2      cash expenses > excluding operator dwellings  303783582000   
3   cash expenses > excluding net rent to landlords  282272137000   
4  current operating > including operator dwellings  256051343000   

           2017          2018          2019          2020          2021  \
0  349660665000  343095241000  347817817000  357238237000  371910340000   
1  345863730000  339374361000  344024660000  353974743000  368811812000   
2  311887235000  311369803000  317313945000  326478643000  345403052000   
3  292702830000  294655691000  299280289000  307254818000  325819001000   
4  264285840000  264219887000  268132721000  276155461000  294338410000   

           2022          2023          2024          2025  
0  435750661000  461884765000  452870646000  450372554000  
1  43195350400

In [31]:
# Save cleaned data
outputFile = 'cleanProductionExpensesData.xlsx'
with pd.ExcelWriter(outputFile, engine='openpyxl') as writer:
    cleaned_expenseDF.to_excel(writer, index=False, sheet_name='Cleaned Data')
